# Use Deeptime to establish EDMD

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

from deeptime.basis import Monomials
from deeptime.decomposition import EDMD

In [2]:
data = np.load('./databygroup.npy',allow_pickle=True).tolist()
TrainData,TestData={},{}
for k in data.keys():
    if ('historydata' in k):
        TrainData[k]=data[k]
    else:
        TestData[k]=data[k]

m = TestData[k][0,0:TestData[k].shape[1]-5].shape[0]
print(m)

19


In [3]:
for k in TestData.keys():
    if os.path.exists('./DLEDMD_model/'+k):
        pass
    else:
        os.makedirs('./DLEDMD_model/'+k)

if os.path.exists('./DLEDMD_model/traindata'):
    pass
else:
    os.makedirs('./DLEDMD_model/traindata')

for d in TestData.keys():
    if not os.path.exists('./results/'+d):
        #if '.csv0' not in d:
        os.makedirs('./results/'+d)
    
    os.chdir(os.path.pardir)
    if not os.path.exists('./Step3_dataselect/results_s/'+d):
        #if '.csv0' not in d:
        os.makedirs('./Step3_dataselect/results_s/'+d)
    os.chdir('./Step2_Koopman')
        
   
if not os.path.exists('./results/traindata'):
    os.makedirs('./results/traindata')
os.chdir(os.path.pardir)
if not os.path.exists('./Step3_dataselect/results_s/traindata'):
    os.makedirs('./Step3_dataselect/results_s/traindata')
os.chdir('./Step2_Koopman')

In [4]:
basis = Monomials(p = 10, d = m)

In [5]:
xtrain = TestData[k][0:10,0:TestData[k].shape[1]-5]
#ytrain = TestData[k][1:11,[-5,-4,-3,-2,-1]]
ytrain = TestData[k][1:11,0:TestData[k].shape[1]-5]
print(xtrain.shape,ytrain.shape)

(10, 19) (10, 19)


In [6]:
m

19

In [8]:
edmd_estimator = EDMD(basis, n_eigs=2)
edmd_model = edmd_estimator.fit((xtrain, ytrain)).fetch_model()

MemoryError: Unable to allocate 2.85 PiB for an array with shape (20030010, 20030010) and data type float64

In [ ]:
plt.plot(np.real(edmd_model.eigenvalues), 'x')
plt.title('Dominant eigenvalues')

# test

In [ ]:
from deeptime.data import triple_well_2d
system = triple_well_2d()

x = np.linspace(-2, 2, num=100)
y = np.linspace(-1, 2, num=100)
XX, YY = np.meshgrid(x, y)
coords = np.dstack((XX, YY)).reshape(-1, 2)
V = system.potential(coords).reshape(XX.shape)

plt.contourf(x, y, V, levels=np.linspace(-4.5, 4.5, 20), cmap='coolwarm');

In [ ]:
traj = system.trajectory(x0=[[-1, 0]], length=200, seed=66)

plt.contourf(x, y, V, levels=np.linspace(-4.5, 4.5, 20), cmap='coolwarm')
plt.plot(*traj.T, c='black', lw=.5)

N = 25000
state = np.random.RandomState(seed=42)
X = np.stack([state.uniform(-2, 2, size=(N,)), state.uniform(-1, 2, size=(N,))]).T
Y = system(X, n_jobs=8)

In [6]:
from deeptime.basis import Monomials

basis = Monomials(p=10, d=2)

In [7]:
from deeptime.decomposition import EDMD

edmd_estimator = EDMD(basis, n_eigs=8)
edmd_model = edmd_estimator.fit((X, Y)).fetch_model()

In [ ]:
plt.plot(np.real(edmd_model.eigenvalues), 'x')
plt.title('Dominant eigenvalues')

In [14]:
N = 25000
state = np.random.RandomState(seed=30)
X1 = np.stack([state.uniform(-2, 2, size=(N,)), state.uniform(-1, 2, size=(N,))]).T
Y1 = system(X1, n_jobs=8)
edmd_estimator1 = EDMD(basis, n_eigs=8)
edmd_model1 = edmd_estimator1.fit((X1, Y1)).fetch_model()

In [ ]:
plt.plot(np.real(edmd_model.eigenvalues), 'x')
plt.title('Dominant eigenvalues')

In [13]:
phi = np.real(edmd_model.transform(X1, propagate=False))

# normalize
for i in range(len(edmd_model.eigenvalues)):
    phi[:, i] = phi[:, i] / np.max(np.abs(phi[:, i]))

In [ ]:
fig = plt.figure(figsize=(8, 8))
gs = fig.add_gridspec(ncols=2, nrows=2)

ax = fig.add_subplot(gs[0, 0])
ax.scatter(*X.T, c=phi[:, 0], cmap='coolwarm')
ax.set_title('1st eigenfunction')

ax = fig.add_subplot(gs[0, 1])
ax.scatter(*X.T, c=phi[:, 1], cmap='coolwarm')
ax.set_title('2nd eigenfunction')

ax = fig.add_subplot(gs[1, 0])
ax.scatter(*X.T, c=phi[:, 2], cmap='coolwarm')
ax.set_title('3rd eigenfunction')

ax = fig.add_subplot(gs[1, 1])
ax.scatter(*X.T, c=phi[:, 3], cmap='coolwarm')
ax.set_title('4th eigenfunction')